# Import Libraries

In [ ]:
# Import Libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from bs4 import BeautifulSoup
from openpyxl import Workbook
from io import BytesIO
from datetime import datetime
import random
import time
import json
import requests
import re
import os
import io
from urllib.parse import urlparse
import csv
import pandas as pd

# Setup ChromeDriver and Open Webpage

In [ ]:
# Setup chromedriver
driver = webdriver.Chrome()

In [ ]:
# Open the Webpage
driver.get('https://reference.medscape.com/drugs')

# Extract html content

In [ ]:
soups = []

# Parse the HTML
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
soups.append(soup)

# Extract urls

In [ ]:
drug_categories_urls = []

for soup in soups:
    elements = soup.find_all('ul',class_ = 'classdruglist')
    # Extract urls from <a> tags inside each <ul>
    for element in elements:
        # Find all <a> tags inside the <ul>
        a_tags = element.find_all('a')

        # Extract the href attribute
        for a_tag in a_tags:
            href = a_tag['href']

            # Validate href attribute
            if href:
                drug_categories_urls.append(href)

[<ul class="classdruglist">
 <li><a href="https://reference.medscape.com/drugs/allergy-cold">Allergy &amp; Cold</a></li>
 <li><a href="https://reference.medscape.com/drugs/anesthetics">Anesthetics</a></li>
 <li><a href="https://reference.medscape.com/drugs/antidotes">Antidotes</a></li>
 <li><a href="https://reference.medscape.com/drugs/antimicrobials">Antimicrobials</a></li>
 <li><a href="https://reference.medscape.com/drugs/blood-components">Blood Components</a></li>
 <li><a href="https://reference.medscape.com/drugs/cardiovascular">Cardiovascular</a></li>
 <li><a href="https://reference.medscape.com/drugs/critical-care">Critical Care</a></li>
 <li><a href="https://reference.medscape.com/drugs/dental-oral-care">Dental &amp; Oral Care</a></li>
 <li><a href="https://reference.medscape.com/drugs/dermatologics">Dermatologics</a></li>
 <li><a href="https://reference.medscape.com/drugs/gastrointestinal">Gastrointestinal</a></li>
 <li><a href="https://reference.medscape.com/drugs/hematologic

# Process urls and extract disease conditions Information

In [ ]:
# Initialize list to store flattened data
flattened_data = []

# Loop through each category URL
for url in drug_categories_urls:
    try:
        print(f"Scraping: {url}")
        driver.get(url)

        # Randomized sleep time (3 to 6 seconds)
        time.sleep(random.uniform(3, 6))

        # Wait for the page to load completely
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )

        # Parse HTML with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Find all sections containing drugs
        category_sections = soup.find_all("div", class_="topic-section")

        for section in category_sections:
            # Extract category name
            category_name = section.find("div", class_="topic-head").text.strip()

            #  Extract drugs within the category
            drug_elements = section.find_all("li")  # Each drug is inside <li>

            for drug in drug_elements:
                drug_name = drug.find("a").text.strip()
                drug_link = drug.find("a")["href"]

                # Append flattened data
                flattened_data.append({
                    'Category': category_name,
                    'Drug Name': drug_name,
                    'Drug Link': drug_link
                })

    except Exception as e:
        print(f"Error scraping {url}: {e}")


Scraping: https://reference.medscape.com/drugs/allergy-cold
Scraping: https://reference.medscape.com/drugs/anesthetics
Scraping: https://reference.medscape.com/drugs/antidotes
Scraping: https://reference.medscape.com/drugs/antimicrobials
Scraping: https://reference.medscape.com/drugs/blood-components
Scraping: https://reference.medscape.com/drugs/cardiovascular
Scraping: https://reference.medscape.com/drugs/critical-care
Scraping: https://reference.medscape.com/drugs/dental-oral-care
Scraping: https://reference.medscape.com/drugs/dermatologics
Scraping: https://reference.medscape.com/drugs/gastrointestinal
Scraping: https://reference.medscape.com/drugs/hematologics
Scraping: https://reference.medscape.com/drugs/herbals-supplements
Scraping: https://reference.medscape.com/drugs/imaging-agents
Scraping: https://reference.medscape.com/drugs/immunologics
Scraping: https://reference.medscape.com/drugs/metabolic-endocrine
Scraping: https://reference.medscape.com/drugs/neurologics
Scraping: h

# Save CSV file.

In [ ]:
# Convert the flattened data into a pandas DataFrame
df = pd.DataFrame(flattened_data)

medscape_csv = df.to_csv('MedScape_Drugs.csv',index = False)

medscape_csv